In [15]:
import random
from deap import base
from deap import creator
from deap import tools
from deap import gp
import array
import numpy
import operator
from deap import gp
import json
import pandas as pd

### INTRODUCTION 
* Dans la régression, on cherche à ajuster les paramètres d’un modèle (linéaire, logistique, ou autre) pour approcher une variable à partir d’autres qui lui sont corrélées. 
* Pourtant, la forme générale du modèle est donnée ou, dit autrement, le modèle est une hypothèse. 
* La régression symbolique, par contre, consiste à chercher un modèle (c-à-d une fonction) qui approche la variable cible à partir des autres, sans faire d’hypothèses sur sa forme. 
* La programmation génétique semble donc un outil parfaitement adapté pour ce genre d’exercice

* #### Nous allons utiliser le framework DEAP pour développer une approche de la régression symbolique basée sur la programmation génétique. 
* Nous supposerons que le jeu de données du problème à résoudre soit donné dans un fichier en format TSV (de l’anlais tab-separated values), dont la première ligne contient les noms des variables et les lignes suivantes contiennent les observations dont ont dispose. 
* La variable à approcher à partir des autres est, par convention, la dernière. Un exemple d’un tel fichier est le suivant :

In [35]:
df = pd.read_csv("data.csv", sep=";")
df

,x1,x2,y
0,"1,00","0,00","1,00"
1,"0,00","1,00","1,00"
2,"0,00","0,00","0,00"
3,"1,00","1,00","2,00"
4,"1,00","2,00","3,00"
5,"2,00","1,00","3,00"


### Consignes
1. Vous pouvez vous inspirer du tutoriel avancé de DEAP sur la programmation génétique.
2. Choisissez un ensemble d’opérations primitives suffisant pour exprimer même les fonctions les plus complexes ; outre aux opérations arithmétiques, pensez à inclure aussi l’exponentiel, le logarithme et des fonctions trigonométriques.
3. Faites attention aux opérations/fonctions dont le domaine de définition ne coincide pas avec R : pour éviter tout problème, redéfinissez-les pour les « protéger » des arguments hors domaine.

In [34]:
pset = gp.PrimitiveSet("main", 2) # crée un ensemble primitif
#ses arguments sont le nom de la procédure qu'elle va générer "main"
#et son nombre d'entrée 2

#ajoute des fonctions comme primitives 
# le premier argument est la fonction à affihcer "max"
# le second est le nombre d'input/argument
pset.addPrimitive(max, 2) 
pset.addPrimitive(operator.add, 2)  
pset.addPrimitive(operator.mul, 2)

#ajoute une constante terminale
pset.addTerminal(3)

#Actuellement, les noms par défaut des arguments sont "ARG0" et "ARG1". 
#Pour les changer en "x" et "y", il suffit d'appeler

pset.renameArguments(ARG0="x1")
pset.renameArguments(ARG1="x2")
#Dans ce cas, toutes les fonctions prennent deux arguments. 

#Une fonction de négation à 1 argument, par exemple, 
#pourrait être réalisée avec :
#pset.addPrimitive(operator.neg, 1)


# Notre ensemble de primitives est maintenant prêt à générer quelques arbres.
# Le module gp contient trois fonctions de génération d'expressions préfixes :
# genFull(), genGrow(), et genHalfAndHalf(). 


AttributeError: 'DataFrame' object has no attribute 'addPrimitive'

In [30]:
# Leur premier argument est un ensemble primitif. 
# Elles retournent une expression préfixe valide sous la forme d'une liste de primitives. 
# Le contenu de cette liste peut être lu par la classe PrimitiveTree pour créer un arbre de préfixes.

In [31]:
expr = gp.genFull(pset, min_=1, max_=3) 
tree = gp.PrimitiveTree(expr)

In [39]:
toolbox = base.Toolbox()
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
pop = toolbox.population(n=300)
pop

AttributeError: 'Toolbox' object has no attribute 'individual'

### Problème de régression symbolique : introduction à la GP
lien : https://deap.readthedocs.io/en/master/examples/gp_symbreg.html


In [41]:
import math

def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))

In [42]:
pset.renameArguments(ARG0='x')

In [43]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [44]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [45]:
def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

In [46]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

In [49]:
import algorithms
pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats, halloffame=hof, verbose=True)

ModuleNotFoundError: No module named 'algorithms'